In [4]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
#load your openai key
os.environ['OPENAI_API_KEY'] = 'your openapi key here'

In [ ]:
llm=OpenAI(temperature=0.9,max_tokens=500)

In [ ]:
#load the data
loaders=UnstructuredURLLoader(urls=[
    "add your urls"])
data=loaders.load()


In [ ]:
#now split the data into chunks
splitting_text=RecursiveCharacterTextSplitter(
    chunk_size=1000)
docs=splitting_text.split_documents(data)

In [ ]:
#Create embeddings for these chunks and save them to FAISS index
embeddings=OpenAIEmbeddings()
#passing the documents to create FAISS vector index
vectorindex_openai=FAISS.from_documents(docs,embeddings)

In [ ]:
#storing the vector index in local
file_path="your_file_path.pkl"
with open(file_path,"wb") as f:
    pickle.dump(vectorindex_openai,f)
    
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)


In [ ]:
#Retrieve similar embeddings for a given question and call LLM to retrieve final answer
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

In [ ]:
#to find the answer for a query
query = "asking a question from the specified urls to retrieve the answer "


langchain.debug=True

chain({"question": query}, return_only_outputs=True)